In [1]:
# coding: UTF-8
#! /usr/bin/env python
from numpy import *
from matplotlib.font_manager import FontProperties
from matplotlib.pyplot import figure, show
import matplotlib.pyplot as plt
import csv
import sys
%matplotlib inline
import matplotlib

[origin](https://gist.github.com/northface/4577328#file-data-txt)

In [2]:
csvfile = open('data.txt')
dlist = []
slist = []
tmp = []
hlist = []
if csvfile != None:
    n = 0
    for row in csv.reader(csvfile):
        for col in row:
            Lt = col.strip().split('\t')
            if n == 0:
                tmp = Lt
                hlist = tmp[1:]
            else:
                cols = Lt
                slist.append(cols[0])
                dlist.append([float(str) for str in cols[1:]])
            n += 1
csvfile.close()

In [3]:
HN = vstack(hlist)
SN = vstack(slist)
R = vstack(dlist)
print('--分析対象となるデータ行列:R--')
print(R)

R_num_row = len(R)
R_num_col = size(R[0])
S = [0 for i in range(R_num_row)]
C = [0 for i in range(R_num_col)]

--分析対象となるデータ行列:R--
[[ 0.  0.  1.  1.  1.  0.  0.]
 [ 1.  1.  0.  1.  0.  1.  0.]
 [ 1.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.  1.]]


In [4]:
print('--カテゴリごとの点数が対角にある行列の逆行列:CT--')
for m in range(0, R_num_col):
    sum = 0
    for n in range(0, R_num_row):
        sum += R[n,m]
    C[m] = sum
CI = mat(diag(C)).I
print(CI)

--カテゴリごとの点数が対角にある行列の逆行列:CT--
[[ 0.33333333  0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.          0.          0.          0.        ]
 [ 0.          0.          1.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.33333333  0.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          1.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.33333333]]


In [5]:
print('--サンプルごとの点数の平方根が対角にある行列の逆行列:SI--')
for n in range(0, R_num_row):
    sum = 0
    for m in range(0, R_num_col):
        sum += R[n,m]
    S[n] = sqrt(sum)

SI = mat(diag(S)).I
print(SI)

--サンプルごとの点数の平方根が対角にある行列の逆行列:SI--
[[ 0.57735027  0.          0.          0.          0.        ]
 [ 0.          0.5         0.          0.          0.        ]
 [ 0.          0.          0.57735027  0.          0.        ]
 [ 0.          0.          0.          0.57735027  0.        ]
 [ 0.          0.          0.          0.          0.70710678]]


In [6]:
print("--データ行列Rの転置行列:RT--")
RT = R.T
print(RT)

--データ行列Rの転置行列:RT--
[[ 0.  1.  1.  1.  0.]
 [ 0.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.]
 [ 1.  1.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.]]


In [7]:
print('--SI*R*CI*RT*SI*v=r2*vを満たす固有値・固有ベクトルを求める--')
X = SI*R*CI*RT*SI
v, r2, vh = linalg.svd(X)
print('固有値:r2')
print(r2)

--SI*R*CI*RT*SI*v=r2*vを満たす固有値・固有ベクトルを求める--
固有値:r2
[ 1.          0.7460774   0.32168052  0.17281115  0.03720871]


In [8]:
print('寄与率')
r2 = r2[1:]
r3 = r2
print(r3)
for n in range(0, size(r3)):
    r3[n] = r3[n] / r3.sum()
print(r3)
print()

print('固有ベクトル:v')
print(v)
print()

print('単相関係数:r')
r = sqrt(r2)
print(r)

寄与率
[ 0.7460774   0.32168052  0.17281115  0.03720871]
[ 0.58388666  0.28835089  0.15967657  0.03480302]

固有ベクトル:v
[[-0.4472136   0.87246258 -0.1549608   0.1084679  -0.05505377]
 [-0.51639778 -0.15397441  0.7930122   0.2762141   0.06680293]
 [-0.4472136  -0.33287065 -0.29635376  0.01669324 -0.77530182]
 [-0.4472136  -0.11557079 -0.05067395 -0.82978354  0.30908706]
 [-0.36514837 -0.30156501 -0.50668076  0.47235696  0.54394735]]

単相関係数:r
[ 0.76412477  0.53698314  0.39959551  0.18655569]


In [9]:
print('--サンプルスコア--')
x1 = sqrt(R.sum())*v[:,1:]
x1 = SI*x1
print(x1)
print()

print('--カテゴリスコア--')
y1 = CI*R.T*x1
y2 = zeros((len(y1),size(y1[0])))
for j in range(0,size(y1[0])):
  y2[:,j:j+1] = y1[:,j]/r[j]
y2 = mat(y2)
print(y2)

--サンプルスコア--
[[ 1.95088563 -0.34650288  0.24254159 -0.12310398]
 [-0.29817016  1.53566153  0.5348863   0.12936331]
 [-0.74432141 -0.66266715  0.03732722 -1.73362758]
 [-0.25842415 -0.1133104  -1.85545241  0.69113969]
 [-0.82586979 -1.38760242  1.2936028   1.48966118]]

--カテゴリスコア--
[[-0.56749708  0.47157532 -1.07044821 -1.63154962]
 [-0.81503328 -0.31944394  1.55642084 -0.20477011]
 [ 2.55309828 -0.64527702  0.60696776 -0.65987793]
 [ 0.60823022  0.6678349  -0.89926313  1.24609624]
 [ 2.55309828 -0.64527702  0.60696776 -0.65987793]
 [-0.39021136  2.85979468  1.33856935  0.69343002]
 [-0.79769492 -1.34304649 -0.43754445  0.79899876]]


[matplot参考](http://kaisk.hatenadiary.com/entry/2015/02/15/215831)

In [11]:
print('--散布図表示 (第一成分X軸・第二成分Y軸) --')
s_score_x = zeros((1,1))
s_score_y = zeros((1,1))
c_score_x = zeros((1,1))
c_score_y = zeros((1,1))

s_score_x = array(x1[:,0])
s_score_y = array(x1[:,1])
c_score_x = array(y2[:,0])
c_score_y = array(y2[:,1])

fig1 = plt.figure()

ax = fig1.add_subplot(111, autoscale_on=False, xlim=(-1,5), ylim=(-4,3))
l, = plt.plot([], [], 'r-')

fig1 = plt.scatter(s_score_x,s_score_y, marker='o', edgecolors = 'r', facecolor='None', s=40, alpha = 1, label = "Sample Score")
fig1 = plt.scatter(c_score_x,c_score_y, marker='D', edgecolors = 'b', facecolor='None', s=40, alpha = 1, label = "Category Score")

fp = FontProperties(fname=r'/home/fukami/.pyenv/versions/3.5.2/envs/pyenv3.5.2/lib/python3.5/site-packages/matplotlib/mpl-data/fonts/ttf/ipaexg.ttf ')

for n in range(0, size(s_score_x)):
    SN[n]
    ax.annotate(
            slist[n],
            xy=(s_score_x[n]-0.05, s_score_y[n]),
            xytext=(s_score_x[n]-0.2, s_score_y[n]),
            arrowprops=dict(
                arrowstyle='-',
                facecolor='black',
                ),
            horizontalalignment='right', verticalalignment='center',
            fontproperties=fp
    )
print(hlist)
for n in range(0, size(c_score_x)):
    ax.annotate(
            hlist[n],
            xy=(c_score_x[n]+0.05, c_score_y[n]),
            xytext=(c_score_x[n]+0.2, c_score_y[n]),
            arrowprops=dict(
                arrowstyle='-',
                facecolor='black',
            ),
        horizontalalignment='left', verticalalignment='center',
        fontproperties=fp
    )

plt.xlim(-3,3)
plt.ylim(-3,3)
plt.axhline()
plt.axvline()

plt.grid(True)
plt.legend(loc='best')
plt.xlabel('成分1', fontproperties=fp)
plt.ylabel('成分2', fontproperties=fp)
plt.title('数量化三類', fontproperties=fp)
plt.show()

--散布図表示 (第一成分X軸・第二成分Y軸) --
['KK', 'nana', 'mommo', 'Lay', '高校生', '大学生', '会社員']


/home/fukami/.pyenv/versions/3.5.2/envs/pyenv3.5.2/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


FileNotFoundError: [Errno 2] No such file or directory: '/home/fukami/.pyenv/versions/3.5.2/envs/pyenv3.5.2/lib/python3.5/site-packages/matplotlib/mpl-data/fonts/ttf/ipaexg.ttf '

In [18]:
print(sys.argv[0])

/home/fukami/.pyenv/versions/3.5.2/envs/pyenv3.5.2/lib/python3.5/site-packages/ipykernel/__main__.py
